In [ ]:
import subprocess

def ensure_t4_gpu():
    # Check current GPU type
    gpu_info = subprocess.run(["nvidia-smi", "--query-gpu=gpu_name", "--format=csv,noheader"], capture_output=True, text=True)
    gpu_name = gpu_info.stdout.strip()

    if "T4" in gpu_name:
        print(f"✅ T4 GPU is already assigned: {gpu_name}")
    else:
        print(f"⚠️ Current GPU is {gpu_name}. Restarting runtime for T4 assignment...")
        from google.colab import runtime
        runtime.unassign()  # Force runtime restart

# Run the function
ensure_t4_gpu()


✅ T4 GPU is already assigned: Tesla T4


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatib

In [ ]:
import torch
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import json
from sklearn.model_selection import train_test_split

In [ ]:
# Load and prepare dataset
with open('total_normalized_data.json', 'r') as file:
    dataset = json.load(file)

dataset = Dataset.from_dict({
    key: [item[key] for item in dataset]
    for key in dataset[0].keys()
})

In [ ]:
# Split dataset
dataset_dict = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_val_split = dataset_dict['train'].train_test_split(test_size=0.111, shuffle=True, seed=42)

dataset = DatasetDict({
    'train': train_val_split['train'],
    'validation': train_val_split['test'],
    'test': dataset_dict['test']
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'entities', 'label'],
        num_rows: 2222
    })
    validation: Dataset({
        features: ['text', 'intent', 'entities', 'label'],
        num_rows: 278
    })
    test: Dataset({
        features: ['text', 'intent', 'entities', 'label'],
        num_rows: 625
    })
})

In [ ]:
dataset["train"][1]

{'text': 'Update the reminder to book the conference room on October fifth atomic number 85 I PM.',
 'intent': 'update_remainder',
 'entities': [{'date': None,
   'time': None,
   'title': 'book the conference room'},
  {'date': 'October 5th', 'time': None, 'title': None},
  {'date': None, 'time': '1 PM', 'title': None}],
 'label': 'update_remainder'}

In [ ]:
# Map intents to numerical labels
unique_intents = list(set(dataset["train"]["intent"]))
intent_to_label = {intent: idx for idx, intent in enumerate(unique_intents)}

def map_intent_to_label(example):
    example["label"] = intent_to_label[example["intent"]]
    return example

dataset = dataset.map(map_intent_to_label)

Map:   0%|          | 0/2222 [00:00<?, ? examples/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

In [ ]:
intent_to_label

{'add_task': 0,
 'add_remainder': 1,
 'update_remainder': 2,
 'delete_remainder': 3,
 'delete_task': 4,
 'update_task': 5}

In [ ]:
# Tokenize dataset
MODEL_NAME = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2222 [00:00<?, ? examples/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

In [ ]:
# Split into train, validation, and test sets
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

In [ ]:
# Initialize model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(unique_intents)
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Define accuracy metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.348400,0.204196,0.938849
2,0.110600,0.138377,0.953237
3,0.102300,0.144487,0.949640


TrainOutput(global_step=417, training_loss=0.3036820813477468, metrics={'train_runtime': 265.7264, 'train_samples_per_second': 25.086, 'train_steps_per_second': 1.569, 'total_flos': 438490321302528.0, 'train_loss': 0.3036820813477468, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate(test_dataset)
print("Test Results:", results)

Test Results: {'eval_loss': 0.28592729568481445, 'eval_accuracy': 0.9152, 'eval_runtime': 4.4886, 'eval_samples_per_second': 139.243, 'eval_steps_per_second': 8.912, 'epoch': 3.0}


In [ ]:
trainer.save_model("./results")
tokenizer.save_pretrained("./results")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [ ]:
# Load the saved model and tokenizer
MODEL_NAME = "./results"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [ ]:
label_to_intent  = {v: k for k, v in intent_to_label.items()}

def predict_intent(text, model, tokenizer):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class


text = "set an alram for 8 pm tomorrow"
predicted_class = predict_intent(text, model, tokenizer)
print(f"Predicted Intent Class: {label_to_intent[predicted_class]}")

Predicted Intent Class: add_task


In [25]:
import shutil

folder_path = '/content/results'
shutil.make_archive('results', 'zip', folder_path)


'/content/results.zip'

In [ ]:
from google.colab import files

files.download('results.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>